In [8]:
from langchain.llms import GooglePalm

api_key='AIzaSyBkZcLNvR_SZ0XtQ3hiqVF1mHdN7Z-K_k'#(Not the Real Key)

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [9]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

**Oh, samosa, my love**
**You are so delicious, so flaky**
**With your filling of potatoes and peas**
**I could eat you every day**


In [10]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

In [11]:
#%pip install --upgrade langchain langchain-google-vertexai

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

- Now to select embeddings you can use anyone from the Langchain Embeddings Documentation and se its implementation 
- Also to see its effectiveness you can use cosine similary 

In [13]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer


max_seq_length  512


In [21]:
# Checkng the Cosine-similary to see its accuracy 
# Making the list into a Numpy Array and then Reshaping 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming instructor_embeddings.embed_query returns lists of embeddings
sentences_a = "The Federal Reserve on Wednesday raised its benchmark interest rate."
sentences_b = "The funds rose less than 0.5 per cent on Friday"
embeddings_a = np.array(instructor_embeddings.embed_query(sentences_a)).reshape(1, -1)
embeddings_b = np.array(instructor_embeddings.embed_query(sentences_b)).reshape(1, -1)

similarities = cosine_similarity(embeddings_a, embeddings_b)
print(similarities)


[[0.81030708]]


In [20]:
print(len(e))
e[:5]


768


[-0.0438980758190155,
 0.007685545366257429,
 -0.009231900796294212,
 0.024496251717209816,
 0.03359227254986763]

In [ ]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [ ]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [ ]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [ ]:
chain('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you prov